# Top 150 - Stats

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [3]:
# hack to get around certificate issues -- NEED TO FIX
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [4]:
url = 'https://www.pro-football-reference.com/years/2021/fantasy.htm'
df = pd.read_html(url, header=1)[0]

In [5]:
df

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,TD.3,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,1,Derrick Henry,TEN,RB,27,6,6,0,0,0,...,10,1,NaN,154,170.1,176.1,162.1,102,1,1
1,2,Cooper Kupp,LAR,WR,28,6,6,0,0,0,...,7,NaN,NaN,107,152.8,155.8,129.8,55,1,2
2,3,Austin Ekeler,LAC,RB,26,6,6,0,0,0,...,7,1,NaN,102,128.8,135.8,115.3,49,2,3
3,4,Ezekiel Elliott,DAL,RB,26,6,6,0,0,0,...,6,NaN,NaN,99,114.6,120.6,106.6,46,3,4
4,5,Jonathan Taylor,IND,RB,22,6,6,0,0,0,...,5,NaN,NaN,98,113.2,119.2,105.7,46,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,528,Colt McCoy,ARI,QB,35,2,0,0,0,0,...,0,NaN,NaN,-1,-0.5,-0.5,-0.5,NaN,56,NaN
545,529,J.J. Taylor,NWE,RB,23,3,0,0,0,0,...,0,NaN,NaN,-1,1.8,2.8,0.3,NaN,136,NaN
546,530,Trenton Cannon,2TM,RB,27,5,0,0,0,0,...,0,NaN,NaN,-2,-1.6,-0.6,-1.6,NaN,137,NaN
547,531,John Wolford,LAR,QB,26,1,0,0,2,0,...,0,NaN,NaN,-2,-2.1,-1.1,-1.1,NaN,58,NaN


In [6]:
# filter out headers (every 30 rows)
df = df[df.Player!='Player']
# remove NA positions
df = df[df['FantPos'].notna()]
# keep top 150 players
df = df[:150]

In [7]:
# rename columns
passing = {i:'Pass_'+i for i in ('Cmp', 'Att', 'Yds', 'TD', 'Int')}
rushing = {'Att.1':'Rush_Att', 'Yds.1':'Rush_Yds', 'Y/A':'Rush_Y/A', 'TD.1':'Rush_TD'}
receiving = {'Yds.2':'Rec_Yds', 'TD.2':'Rec_TD'}
df.rename(columns = {**passing,**rushing,**receiving,'FantPos':'Pos'}, inplace=True)

In [8]:
# coerce 2PM and 2PP
df['2PM'] = df['2PM'].fillna(0).astype(int) + df['2PP'].fillna(0).astype(int)

In [9]:
df.drop(['Rk','TD.3','2PP','FantPt','PPR','DKPt','VBD','OvRank'], axis=1, inplace=True)

In [10]:
# convert dtypes and fillna w/ appropriate values
# string
df[['Player','Tm','Pos']] = df[['Player','Tm','Pos']].astype('string')
# float
df[["Rush_Y/A",'Y/R','FDPt']] = df[["Rush_Y/A",'Y/R','FDPt']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# integers
df[list(df.select_dtypes(['object']))] = df[list(df.select_dtypes(['object']))].fillna(0).astype(int)

In [11]:
# order by Fantasy Points; use index as 'Overall Rank'
df.sort_values('FDPt', ascending=False, inplace=True)
df.reset_index(inplace=True, drop=True)
df.index = df.index + 1
# reset PosRank by FDPt
df['PosRank'] = df.groupby('Pos')['FDPt'].rank("dense", ascending=False).astype(int)

In [12]:
df

,Player,Tm,Pos,Age,G,GS,Pass_Cmp,Pass_Att,Pass_Yds,Pass_TD,...,Tgt,Rec,Rec_Yds,Y/R,Rec_TD,Fmb,FL,2PM,FDPt,PosRank
1,Derrick Henry,TEN,RB,27,6,6,0,0,0,0,...,18,16,138,8.63,0,0,0,1,162.1,1
2,Patrick Mahomes,KAN,QB,26,6,6,167,242,1887,18,...,0,0,0,0.00,0,2,1,0,161.9,1
3,Josh Allen,BUF,QB,25,6,6,149,230,1723,15,...,0,0,0,0.00,0,3,1,1,159.3,2
4,Tom Brady,TAM,QB,44,6,6,183,267,2064,17,...,0,0,0,0.00,0,2,1,0,155.3,3
5,Kyler Murray,ARI,QB,24,6,6,144,195,1741,14,...,0,0,0,0.00,0,6,0,0,151.2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Tyrod Taylor,HOU,QB,32,2,2,31,44,416,3,...,0,0,0,0.00,0,1,0,0,40.1,32
147,Michael Carter,NYJ,RB,22,5,3,0,0,0,0,...,14,9,64,7.11,0,0,0,0,39.4,45
148,Donovan Peoples-Jones,CLE,WR,22,6,3,0,0,0,0,...,15,13,228,17.54,2,1,1,0,39.3,51
149,Mo Alie-Cox,IND,TE,28,6,2,0,0,0,0,...,18,10,152,15.20,3,0,0,0,38.2,16
